<a href="https://colab.research.google.com/github/dpdusi/Face_Detection/blob/main/Face_Recognition_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:

!pip install ultralytics==8.0.20
!pip install deepface

import os
import cv2
import numpy as np
from ultralytics import YOLO
from deepface import DeepFace
import pickle  # For saving embeddings and labels

# Load the YOLO model
model = YOLO("/content/YoloV8best.pt")  # Replace with the path to your trained YOLO model

# Function to generate embeddings from the cropped face using DeepFace
def get_face_embedding(face_image):
    embedding = DeepFace.represent(face_image, model_name='Facenet', enforce_detection=False)[0]['embedding']
    return embedding

# Dictionary to store embeddings and labels
face_database = {}

# Path to the folder containing input images
input_folder = '/content/Dataset'  # Replace with the path to your input folder

# Path to the folder to save processed images
output_folder = '/content/processed_images'
os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

# Loop through all images in the folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):  # Filter for image files
        image_path = os.path.join(input_folder, filename)

        # Read the image
        image = cv2.imread(image_path)

        # Extract the label from the image filename (e.g., "abc.jpg" -> "abc")
        label = os.path.splitext(filename)[0]

        # Predict faces using the trained YOLO model
        results = model.predict(image_path)

        # Loop through each detected face in the results
        for result in results[0].boxes.data:  # result[0] contains the detection result
            x1, y1, x2, y2 = result[:4]  # Bounding box coordinates (x1, y1, x2, y2)

            # Crop the face from the image using the bounding box
            face_image = image[int(y1):int(y2), int(x1):int(x2)]

            # Optionally, resize the face image if needed for the model
            face_image = cv2.resize(face_image, (160, 160))  # Example for FaceNet or similar models

            # Get the embedding for the cropped face
            embedding = get_face_embedding(face_image)

            # Add the embedding and label to the face database
            face_database[label] = embedding

            # Optionally, print the processed embeddings
            print(f"Processed {label}: Embedding added.")

            # Draw bounding box and label on the image
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(image, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Save the processed image to the output folder
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, image)
        print(f"Saved processed image: {output_path}")

# Save the face database for future use
with open("face_database.pkl", "wb") as file:
    pickle.dump(face_database, file)

print("Face database saved successfully!")


Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


Processed Anusha: Embedding added.
Saved processed image: /content/processed_images/Anusha.jpg
Processed Monica: Embedding added.
Saved processed image: /content/processed_images/Monica.jpg
Processed Gayatri: Embedding added.
Saved processed image: /content/processed_images/Gayatri.jpg
Processed Karthik: Embedding added.
Saved processed image: /content/processed_images/Karthik.jpg
Processed Prasad: Embedding added.
Saved processed image: /content/processed_images/Prasad.jpg
Face database saved successfully!


In [9]:
import pickle

# Load the pickle file
with open("face_database.pkl", "rb") as file:
    face_database = pickle.load(file)

# Check the number of entries
print(f"Number of entries in the pickle file: {len(face_database)}")

# Optional: Print the keys (labels) to inspect the data
print("Labels in the pickle file:", list(face_database.keys()))


Number of entries in the pickle file: 5
Labels in the pickle file: ['Anusha', 'Monica', 'Gayatri', 'Karthik', 'Prasad']


In [20]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from deepface import DeepFace
from sklearn.metrics.pairwise import cosine_similarity
import pickle  # For saving embeddings and labels

# Load the YOLO model
model = YOLO("/content/YoloV8best.pt")  # Replace with the path to your trained YOLO model

# Function to generate embeddings from the cropped face using DeepFace
def get_face_embedding(face_image):
    embedding = DeepFace.represent(face_image, model_name='Facenet', enforce_detection=False)[0]['embedding']
    return embedding

# Load the face database
with open("face_database.pkl", "rb") as file:
    face_database = pickle.load(file)

print("Face database loaded successfully!")

# Path to the new image for recognition
new_image_path = '/content/Karthik.jpg'  # Replace with the path to your new image

# Read the new image
new_image = cv2.imread(new_image_path)

# Predict faces in the new image
results = model.predict(new_image_path)

# Loop through each detected face in the results
for result in results[0].boxes.data:
    x1, y1, x2, y2 = result[:4]  # Bounding box coordinates (x1, y1, x2, y2)

    # Crop the face from the new image
    face_image = new_image[int(y1):int(y2), int(x1):int(x2)]

    # Optionally, resize the face image if needed for the model
    face_image = cv2.resize(face_image, (160, 160))  # Example for FaceNet or similar models

    # Generate the embedding for the cropped face
    new_embedding = get_face_embedding(face_image)

    # Initialize variables to track the best match
    max_similarity = -1
    recognized_label = "Unknown"
    similarity_threshold = 0.3  # Adjust this value based on your requirements

    # Compare the new embedding with stored embeddings
    for label, stored_embedding in face_database.items():
        # Ensure stored embedding is a 1D array
        stored_embedding = np.array(stored_embedding).flatten()

        # Compute cosine similarity
        similarity = cosine_similarity(
            np.array(new_embedding).reshape(1, -1),  # Reshape new embedding to 2D
            stored_embedding.reshape(1, -1)         # Reshape stored embedding to 2D
        )[0][0]

        # Update the best match if the similarity is higher
        if similarity > max_similarity and similarity > similarity_threshold:
            max_similarity = similarity
            recognized_label = label

    # Draw bounding box and recognized label on the new image
    cv2.rectangle(new_image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    cv2.putText(new_image, recognized_label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    print(f"Recognized: {recognized_label} with similarity: {max_similarity}")

# Display or save the new image with recognition results
output_path = '/content/recognized_new_image.jpg'
cv2.imwrite(output_path, new_image)
print(f"Processed image with recognition saved: {output_path}")


Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


Face database loaded successfully!
Recognized: Karthik with similarity: 0.3978705090729182
Recognized: Unknown with similarity: -1
Processed image with recognition saved: /content/recognized_new_image.jpg


In [4]:
import cv2

# Replace with your camera credentials and IP address
username = "admin"
password = "smart%40123"
camera_ip = "202.53.85.100"
port = "8888"
camera_path = "live"  # Adjust the path based on your camera's documentation

# Construct the URL with authentication
camera_url = f"http://{username}:{password}@{camera_ip}:{port}/{camera_path}"

# Access the video feed
cap = cv2.VideoCapture(camera_url)

if not cap.isOpened():
    print("Error: Cannot access the camera feed")
else:
    print("Camera feed accessed successfully")

# Read and display frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error: Cannot read frame from the camera feed")
        break

    # Display the frame
    cv2.imshow('Camera Feed', frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Error: Cannot access the camera feed


In [5]:
import cv2
import urllib.parse

# Replace with your camera credentials and IP address
username = "admin"
password = "smart@123"  # Password with @ symbol
camera_ip = "202.53.85.100"
port = "8888"
path = "live"

# URL-encode the username and password
encoded_password = urllib.parse.quote(password)

# Construct the URL with authentication
#camera_url = f"http://{username}:{encoded_password}@{camera_ip}:{port}/"
camera_url = f"http://{username}:{password}@{camera_ip}:{port}/{camera_path}"


# Access the video feed
cap = cv2.VideoCapture(camera_url)

if not cap.isOpened():
    print("Error: Cannot access the camera feed")
else:
    print("Camera feed accessed successfully")


Error: Cannot access the camera feed


In [7]:
import cv2
import numpy as np
import requests
from requests.auth import HTTPBasicAuth

# Replace with your camera credentials and IP address
username = "admin"
password = "smart@123"
camera_url = "http://202.53.85.100:8888"  # Simplified URL without credentials

# Function to fetch video frames with authentication
def get_frame():
    response = requests.get(camera_url, auth=HTTPBasicAuth(username, password), stream=True)
    if response.status_code == 200:
        # Convert the response content into a numpy array for OpenCV
        arr = np.asarray(bytearray(response.content), dtype=np.uint8)
        frame = cv2.imdecode(arr, cv2.IMREAD_COLOR)
        return frame
    else:
        print("Failed to access the video feed. Status code:", response.status_code)
        return None

# Display the live feed
while True:
    frame = get_frame()
    if frame is not None:
        cv2.imshow("Live Feed", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
            break
    else:
        print("No frame received.")

cv2.destroyAllWindows()

No frame received.
No frame received.
No frame received.
No frame received.
No frame received.
No frame received.
No frame received.
No frame received.
No frame received.
No frame received.
No frame received.


KeyboardInterrupt: 